## Day 7: Recursive Circus

Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:

    pbga (66)
    xhth (57)
    ebii (61)
    havc (66)
    ktlj (57)
    fwft (72) -> ktlj, cntj, xhth
    qoyq (66)
    padx (45) -> pbga, havc, qoyq
    tknk (41) -> ugml, padx, fwft
    jptl (61)
    ugml (68) -> gyxo, ebii, jptl
    gyxo (61)
    cntj (57)

...then you would be able to recreate the structure of the towers that looks like this:

                    gyxo
                  /     
             ugml - ebii
           /      \     
          |         jptl
          |        
          |         pbga
         /        /
    tknk --- padx - havc
         \        \
          |         qoyq
          |             
          |         ktlj
           \      /     
             fwft - cntj
                  \     
                    xhth

In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

In [1]:
import re

node_matcher = re.compile(r'(?P<name>\w+) \((?P<weight>\d+)\)').match

In [2]:
class Node:
    def __init__(self, name, weight, children=None):
        self.name, self.weight, self.children = name, weight, []
        if children:
            self.children = children
            
    def __repr__(self):
        return "Node({}, {}, {})".format(
            repr(self.name),
            repr(self.weight),
            repr(self.children)
        )

In [3]:
def parse_input(lines):
    nodes = {}

    for line in lines:
        parts = line.strip().split(' -> ')
        name, weight = node_matcher(parts[0]).groups()
        weight = int(weight)
        
        node = Node(name, weight)
        
        if len(parts) > 1:
            node.children = parts[1].split(', ')
        
        nodes[name] = node
    
    for name, node in nodes.items():
        for i, child in enumerate(node.children):
            node.children[i] = nodes[child]
    
    return nodes

In [4]:
test_input = """pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)"""

In [5]:
def build_tree(nodes):    
    while len(nodes) > 1:
        remove = set()
        
        for name, node in nodes.items():
            if not node.children:
                remove.add(name)
            else:
                for child in node.children:
                    if child.name in nodes:
                        remove.add(child.name)

        for name in remove:
            del(nodes[name])

    return nodes[next(iter(nodes))]

In [6]:
test_nodes = parse_input(test_input.split('\n'))
test_root = build_tree(test_nodes)
assert test_root.name == 'tknk'
test_root

Node('tknk', 41, [Node('ugml', 68, [Node('gyxo', 61, []), Node('ebii', 61, []), Node('jptl', 61, [])]), Node('padx', 45, [Node('pbga', 66, []), Node('havc', 66, []), Node('qoyq', 66, [])]), Node('fwft', 72, [Node('ktlj', 57, []), Node('cntj', 57, []), Node('xhth', 57, [])])])

Now: let's check the input:

In [7]:
node_list = parse_input(open('input/day7.txt'))
root = build_tree(node_list)
print(root.name)

bsfpjtc


## Part Two

The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.

In the example above, this means that for ugml's disc to be balanced, gyxo, ebii, and jptl must all have the same weight, and they do: 61.

However, for tknk to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same:

    ugml + (gyxo + ebii + jptl) = 68 + (61 + 61 + 61) = 251
    padx + (pbga + havc + qoyq) = 45 + (66 + 66 + 66) = 243
    fwft + (ktlj + cntj + xhth) = 72 + (57 + 57 + 57) = 243

As you can see, tknk's disc is unbalanced: ugml's stack is heavier than the other two. Even though the nodes above ugml are balanced, ugml itself is too heavy: it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced. If this change were made, its weight would be 60.

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

In [8]:
def node_print(node, indent=0):
    print("{}{}".format(' ' * indent, node.weight))
    for child in node.children:
        node_print(child, indent + 2)

In [9]:
def sum_weights(node):
    return node.weight + sum(sum_weights(c) for c in node.children)

In [10]:
from collections import defaultdict

def find_unbalanced_node(node):
    if not node.children:
        return
    
    sums = defaultdict(list)
    for child in node.children:
        sums[sum_weights(child)].append(child)
        
    if len(sums) > 1:
        # Get least common value
        sorted_values = sorted(sums.values(), key=len)
        
        # lc = least common, lcv = least common value
        # kind of stupid to sum the weights again, but meh
        lc = sorted_values[0][0]
        lcv = sum_weights(lc)
        mc = sorted_values[1][0]
        mcv = sum_weights(mc)
        
        res = find_unbalanced_node(lc)
        if res:
            return res
        return abs(lc.weight - (lcv - mcv))

find_unbalanced_node(root)

529